In [25]:
import pandas as pd
from google.cloud import translate
import os
import tqdm

In [6]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="/Users/antoine/git/mtsamples-fr/mtsamples-26c9c13c50e4.json"

In [12]:
dt = pd.read_csv('data/mtsamples.csv')

In [13]:
dt.transcription.iloc[0]

'SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear.,Neck:  Supple without adenopathy.,

In [7]:
client = translate.TranslationServiceClient()

In [8]:
project_id = "mtsamples"
location="global"
parent = f"projects/{project_id}/locations/{location}"

In [36]:

translated = []
for text in tqdm.tqdm(dt.transcription.values):
    if not pd.isna(text) and text is not None :
        response = client.translate_text(
            request={
                "parent": parent,
                "contents": [text],
                "mime_type": "text/plain",  # mime types: text/plain, text/html
                "source_language_code": "en-US",
                "target_language_code": "fr",
            }
        )
        translated.append(response.translations[0].translated_text)
    else:
        translated.append('')




100%|██████████| 4999/4999 [26:05<00:00,  3.19it/s]  


In [38]:
dt['transcription_translated'] = translated

In [45]:
dt.to_json('data/mtsamples_translated.json',orient='records')